In [83]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')
sns.set(style='darkgrid', font_scale=1.1)

In [84]:
try:
    df = pd.read_csv('padisumatera.csv')
    print("Berhasil membaca csv")
except Exception as e:
    print("Gagal membaca data")

Berhasil membaca csv


In [85]:
df.shape

(224, 7)

In [86]:
df.isna().sum()

Provinsi          0
Tahun             0
Produksi          0
Luas Panen        0
Curah hujan       0
Kelembapan        0
Suhu rata-rata    0
dtype: int64

In [87]:
df.duplicated().sum()

np.int64(0)

In [88]:
X_data = df.drop(columns='Provinsi')
y_data = df['Provinsi']

In [89]:
def stratified_split(X_data, y_data, train_size = 0.8, random_state = 42):
    np.random.seed(random_state)
    
    x_train_list, x_test_list = [], []
    y_train_list, y_test_list = [], []
    
    for class_value in np.unique(y_data):
        class_idx = np.where(y_data == class_value)[0]
        
        idx = np.random.permutation(class_idx)
        split_ratio = int(len(idx) * train_size)
        
        x_train_list.append(X_data.iloc[idx[:split_ratio]])
        x_test_list.append(X_data.iloc[idx[split_ratio:]])
        y_train_list.append(y_data.iloc[idx[:split_ratio]])
        y_test_list.append(y_data.iloc[idx[split_ratio:]])
        
    x_train = pd.concat(x_train_list).reset_index(drop =True)
    x_test= pd.concat(x_test_list).reset_index(drop =True)
    y_train = pd.concat(y_train_list).reset_index(drop =True)
    y_test = pd.concat(y_test_list).reset_index(drop =True)
    
    return x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test = stratified_split(X_data, y_data)

In [90]:
def hitung_gini(groups, classes):
    n_instance = float(sum([len(group) for group in groups]))
    gini = 0.0
    
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        
        skor =0.0
        
        for class_val in classes:
            y_group = group.iloc[:, -1]
            p = (y_group == class_val).sum() / size
            skor += p ** 2
            
        gini += (1.0 - skor) * (size / n_instance)
    return gini

In [91]:
def hitung_split(df, feat_idx, threshold):
    
    left = df[df.iloc[:, feat_idx] < threshold]
    right = df[df.iloc[:, feat_idx] >= threshold]
    
    return left, right

In [92]:
def get_best_split(df, classes):
    best_gini = float('inf')
    best_split = {}
    n_features = df.shape[1] - 1
    
    for feat_idx in range(n_features):
        for index, row in df.iterrows():
            groups = hitung_split(df, feat_idx, row.iloc[feat_idx])
            gini = hitung_gini(groups, classes)
            
            if gini < best_gini:
                best_gini = gini
                best_split = {
                    'feat_idx' : feat_idx,
                    'val' : row.iloc[feat_idx],
                    'groups' : groups
                }
    return best_split

In [93]:
def to_terminal(group):
    return group.iloc[:, -1].mode()[0]

def build_tree(df, classes, depth, max_depth):
    root = get_best_split(df, classes)
    if not root:
        return to_terminal(df)
    
    left, right = root['groups']
    del(root['groups'])
    
    if len(left) == 0 or len(right) == 0:
        no_split_data = pd.concat([left, right])
        root['left'] = root['right'] = to_terminal(no_split_data)
        return root
    
    if depth >= max_depth:
        root['left'] = to_terminal(left)
        root['right'] = to_terminal(right)
        return root
    
    root['left'] = build_tree(left, classes, depth + 1, max_depth)
    root['right'] = build_tree(right, classes, depth + 1, max_depth)
    return root

In [94]:
def predict_row(node, row):
    if not isinstance(node, dict):
        return node
    
    if row.iloc[node['feat_idx']] < node['val']:
        return predict_row(node['left'], row)
    else:
        return predict_row(node['right'], row)
    
def predict_batch(tree, df_test):
    predictions = []
    for index, row in df_test.iterrows():
        prediction = predict_row(tree, row)
        predictions.append(prediction)
    return predictions

In [95]:
train_data = pd.concat([x_train, y_train], axis=1)
unique_classes = y_train.unique()
my_tree = build_tree(train_data, unique_classes, depth=0, max_depth=10)
predictions = predict_batch(my_tree, x_test)

In [96]:
accuracy = (y_test == predictions).mean()
accuracy

np.float64(0.6041666666666666)